In [8]:
import json
import GPy
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from os.path import join, splitext
from os import listdir
from multiprocessing import Pool,Lock
import threading
import random

In [9]:
data_path = '/home/rodrigo/Documents/curvas-de-luz-EL4106/OGLE_trainingDB/'


In [10]:
# para tener todos los directorios, bastaria eliminar los que ya se procesaron.
join('asd','fg','ed')

'asd/fg/ed'

In [11]:
def test(lista):
    print( lista[2],'\n' )
l1 = [2,3,4]
l2 = [3,4,5]
l4 = [5,6]

t2 = threading.Thread(target=test, args=[l1])
t2.start()
t2.join()
    

t3 = threading.Thread(target=test, args=[l2])
t3.start()
t3.join()
try:
    t4 = threading.Thread(target=test, args=[l4[2]])
    t4.start()
    t4.join()
except Exception:
    _


4 

5 



In [12]:
lock = Lock()
file = open('database2.csv','w')
def smooth(x, window_len=11, window='hanning'):
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y    

In [13]:
def worker(data,period,file,subclass):
    mjd, mag_r, err_r, mag_b, err_b = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    """
    mjd_b = mjd
    mjd_r = mjd
    """
    mjd_b = mjd[err_b < 9.999]
    mjd_r = mjd[err_r < 9.999]
    mag_b = mag_b[err_b < 9.999]
    mag_r = mag_r[err_r < 9.999]
    err_b = err_b[err_b < 9.999]
    err_r = err_r[err_r < 9.999]
    
    mask = err_b < np.mean(err_b) + 3*np.std(err_b)
    mjd_b = mjd_b[mask]
    mag_b = mag_b[mask]
    err_b = err_b[mask]
    mask = err_r < np.mean(err_r) + 3*np.std(err_r)
    mjd_r = mjd_r[mask]
    mag_r = mag_r[mask]
    err_r = err_r[mask]

    phi = np.mod(mjd_b, period)/period
    I = np.argsort(phi)
    phi_rs = np.linspace(0, 1, num=200)
    mag_b_rs = np.interp(phi_rs, phi[I], mag_b[I])


    kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m = GPy.models.GPHeteroscedasticRegression(phi[:, np.newaxis], mag_b[:, np.newaxis], kernel)
    m['.*het_Gauss.variance'] = err_b[:, np.newaxis]**2 
    m.het_Gauss.variance.fix() 
    #m.het_Gauss.period.fix() 
    
    m.optimize()
    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    mu, var = m._raw_predict(phi_rs[:, np.newaxis])
    # File writer
    mu = json.dumps(mu[:,0].tolist())
    file.write(mu + ';' + subclass+'\n')
    

In [14]:
dir_list = listdir(data_path)
for dir in dir_list:
    for sub_class in listdir(join(data_path,dir)):
        lc_list = [splitext(file)[0] for file in sorted(listdir(join(data_path, dir , sub_class))) if splitext(file)[1] == '.dat']
        if(len(lc_list)>400):
            lc_list=random.sample(lc_list,400)
        print(join(dir,sub_class), len(lc_list))
        if len(lc_list)>50:
            for lc_index in range(1,len(lc_list),4):
                print(lc_index)
                data = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index]+'.time'))
                period = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index]+'.dat'))
                try:
                    data2 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+1]+'.time'))
                    period2 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+1]+'.dat'))
                    t2 = threading.Thread(target=worker, args=[data2,period2,file,join(dir,sub_class)])
                    t2.start()
                    t2.join()
                except Exception:
                    _
                
                try:
                    data3 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+2]+'.time'))
                    period3 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+2]+'.dat'))
                    t3 = threading.Thread(target=worker, args=[data3,period3,file,join(dir,sub_class)])
                    t3.start()
                    t3.join()
                except Exception:
                    _
                try:
                    data4 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+3]+'.time'))
                    period4 = np.loadtxt(join(data_path,dir, sub_class, lc_list[lc_index+3]+'.dat'))
                    t4 = threading.Thread(target=worker, args=[data4,period4,file,join(dir,sub_class)])
                    t4.start()
                    t4.join()
                except Exception:
                    _
                worker(data,period,file,join(dir,sub_class))
            
            
            
        
file.close()
        


    


CEPH/F_10_20 2
CEPH/2O 12
CEPH/1O 400
1
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0004  -1.308273e+03   1.374964e-09 
    00s76  0019  -1.308273e+03   2.009889e-09 
Runtime:     00s76
Optimization status: Converged

Optimization restart 1/5, f = -1308.2728413974194
Optimization restart 2/5, f = -1308.2728413470456
Optimization restart 3/5, f = -1308.2728413697068
Optimization restart 4/5, f = -1308.272841338386
Optimization restart 5/5, f = -1308.272841391267
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s02  0001   8.839630e+01   2.566683e-09 
    00s10  0005   8.839630e+01   2.587191e-09 
    00s22  0013   8.839630e+01   2.576617e-09 
Runtime:     00s22
Optimization status: Converged

Optimization restart 1/5, f = 88.39630176064452
Optimization restart 2/5, f = 88.39630106283857
Optimization restart 3/5, f = 88.39630181663392
Optimization restart 4/5, f = 88.39630113968246

    00s14  0004  -1.304384e+03   1.017925e-10 
Runtime:     00s14
Optimization status: Converged

Optimization restart 1/5, f = -1304.3836891844287
Optimization restart 2/5, f = -1304.3836891560302
Optimization restart 3/5, f = -1304.3836891598473
Optimization restart 4/5, f = -1304.383689175275
Optimization restart 5/5, f = -1304.3836891599208
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000  -1.141331e+03   4.495353e-06 
    00s15  0004  -1.141331e+03   9.849380e-07 
    00s19  0005  -1.141331e+03   9.849380e-07 
Runtime:     00s19
Optimization status: Converged

Optimization restart 1/5, f = -1141.3305383585407
Optimization restart 2/5, f = -1141.3305383464274
Optimization restart 3/5, f = -1141.330538300008
Optimization restart 4/5, f = -1141.3305382430794
Optimization restart 5/5, f = -1141.3305382957244
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s09  0002  -1.398738e+

Optimization restart 2/5, f = -1334.4685975186633
Optimization restart 3/5, f = -1334.4685975130585
Optimization restart 4/5, f = -1334.4685975123878
Optimization restart 5/5, f = -1334.4685975099412
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s05  0002  -1.070035e+03   1.631876e+02 
    00s19  0007  -1.076430e+03   1.005249e-08 
    00s56  0022  -1.076430e+03   1.005249e-08 
Runtime:     00s56
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -1076.4303164796922
Optimization restart 2/5, f = -1076.4303164832934
Optimization restart 3/5, f = -1076.4303164202738
Optimization restart 4/5, f = -1076.430316486546
Optimization restart 5/5, f = -1076.4303164435598
37
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0005  -1.085522e+03   4.993612e-13 
Runtime:     00s15
Optimization status: Converged

Optimization restart 1/5, f = -1085.521551878

Optimization restart 1/5, f = -958.4286513806135
Optimization restart 2/5, f = -958.4286514116425
Optimization restart 3/5, f = -958.428651243582
Optimization restart 4/5, f = -958.4286513242739
Optimization restart 5/5, f = -958.42865123457
53
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000  -1.018853e+03   2.182699e-06 
    00s09  0005  -1.018853e+03   2.602470e-14 
Runtime:     00s09
Optimization status: Converged

Optimization restart 1/5, f = -1018.8531337452819
Optimization restart 2/5, f = -1018.8531337977361
Optimization restart 3/5, f = -1018.8531336833905
Optimization restart 4/5, f = -1018.8531336579836
Optimization restart 5/5, f = -1018.853133643951
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s03  0002  -7.662811e+02   7.914097e+01 
    00s19  0010  -7.738256e+02   4.394466e-07 
    00s40  0022  -7.738256e+02   4.394466e-07 
Runtime:     00s40
Optimization stat

    00s13  0003  -1.423068e+03   5.141923e-05 
    00s17  0004  -1.423068e+03   5.141923e-05 
Runtime:     00s17
Optimization status: Converged

Optimization restart 1/5, f = -1423.067802006981
Optimization restart 2/5, f = -1423.0677706155486
Optimization restart 3/5, f = -1423.06780208559
Optimization restart 4/5, f = -1423.0678020849134
Optimization restart 5/5, f = -1423.06780207821
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0008  -1.190541e+03   6.682785e-09 
    00s31  0013  -1.190541e+03   6.683454e-09 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -1190.5410046554198
Optimization restart 2/5, f = -1190.5410045541678
Optimization restart 3/5, f = -1190.5410046357713
Optimization restart 4/5, f = -1190.5410044576163
Optimization restart 5/5, f = -1190.5410046207787
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s04  0005  -4.809576e+02 

...

    00s11  0003  -3.216499e+02   4.068323e-12 
    00s19  0005  -3.216499e+02   1.264808e-10 
    00s57  0018  -3.216499e+02   1.264829e-10 
Runtime:     00s57
Optimization status: Converged

Optimization restart 1/5, f = -321.6498676169708
Optimization restart 2/5, f = -321.6498675911673
Optimization restart 3/5, f = -321.6498675882749
Optimization restart 4/5, f = -321.64986758307305
Optimization restart 5/5, f = -321.6498676044698
381
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0005  -9.428843e+02   1.147786e-06 
    00s18  0006  -9.428843e+02   1.147836e-06 
    00s53  0022  -9.428843e+02   1.147836e-06 
Runtime:     00s53
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -942.8842978546254
Optimization restart 2/5, f = -942.8842978116604
Optimization restart 3/5, f = -942.8842978131293
Optimization restart 4/5, f = -942.8842978122041
Optimization restart 5/5, f = -942.884297826947
R

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0004  -6.099824e+02   7.496993e-08 
    00s17  0005  -6.099824e+02   7.496993e-08 
Runtime:     00s17
Optimization status: Converged

Optimization restart 1/5, f = -609.9824330616894
Optimization restart 2/5, f = -609.9824330148055
Optimization restart 3/5, f = -609.9824329860136
Optimization restart 4/5, f = -609.9824330128755
Optimization restart 5/5, f = -609.9824330286027
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s09  0003  -6.068147e+02   1.111231e-13 
    00s12  0004  -6.068147e+02   1.111231e-13 
Runtime:     00s12
Optimization status: Converged

Optimization restart 1/5, f = -606.8147009320306
Optimization restart 2/5, f = -606.814700922179
Optimization restart 3/5, f = -606.8147009304158
Optimization restart 4/5, f = -606.8147009291739
Optimization restart 5/5, f = -606.8147006850032
Running L-BFGS-B (Scipy implementatio

    00s06  0002  -3.847602e+02   2.790880e+03 
    00s18  0006  -4.172533e+02   1.171045e-10 
    00s56  0022  -4.172533e+02   1.171045e-10 
Runtime:     00s56
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -417.25329210721
Optimization restart 2/5, f = -407.48212290111087
Optimization restart 3/5, f = -407.48212293872007
Optimization restart 4/5, f = -407.48212293822075
Optimization restart 5/5, f = -407.48212284758233
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0004  -6.623387e+02   1.908480e-09 
    00s24  0007  -6.623387e+02   4.608841e-07 
Runtime:     00s24
Optimization status: Converged

Optimization restart 1/5, f = -662.3386883386108
Optimization restart 2/5, f = -662.3386914768801
Optimization restart 3/5, f = -662.338691555675
Optimization restart 4/5, f = -662.3386917020287
Optimization restart 5/5, f = -662.338691668331
17
Running L-BFGS-B (Scipy implementation) Code:
  r

    00s06  0002  -6.308809e+02   3.558730e-11 
Runtime:     00s06
Optimization status: Converged

Optimization restart 1/5, f = -630.8809299925642
Optimization restart 2/5, f = -630.8809298972585
Optimization restart 3/5, f = -630.8809299844954
Optimization restart 4/5, f = -630.8809299881041
Optimization restart 5/5, f = -630.8809300119203
33
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s09  0003  -1.115914e+03   1.998302e-06 
    00s27  0009  -1.115914e+03   2.849633e-07 
Runtime:     00s27
Optimization status: Converged

Optimization restart 1/5, f = -1115.9135871181434
Optimization restart 2/5, f = -1115.913587022193
Optimization restart 3/5, f = -1115.9135870285459
Optimization restart 4/5, f = -1115.9135870715654
Optimization restart 5/5, f = -1115.9135871018939
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s03  0001  -2.339348e+02   6.869486e-11 
    00s06  0002  -2.339348e+0

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000  -1.128106e+03   2.431006e-08 
    00s11  0003  -1.128106e+03   2.952260e-07 
    00s31  0009  -1.128106e+03   2.949118e-07 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -1128.1059213963363
Optimization restart 2/5, f = -1128.1059213195267
Optimization restart 3/5, f = -1128.1059212650944
Optimization restart 4/5, f = -1128.1059213457283
Optimization restart 5/5, f = -1128.1059212712053
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s07  0002  -7.991688e+02   4.311322e+00 
    00s43  0013  -8.004292e+02   1.899106e-08 
Runtime:     00s43
Optimization status: Converged

Optimization restart 1/5, f = -800.4291644048878
Optimization restart 2/5, f = -800.4291640867684
Optimization restart 3/5, f = -800.4291640728286
Optimization restart 4/5, f = -800.4291643219894
Optimization restart 5/5, f = -80

In [15]:
print("Finished")

Finished
